In [1]:
from sub_func import *

/opt/anaconda3/envs/finTF/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
ticker = '005930'

target_year = '2021'
target_quarter = 'Q1'

start_date = '20210101'
end_date = '20210331'

### 필요 함수 모음

In [16]:
# 감성 점수 및 라벨 기반 필터링 함수
def extract_sentiment_score(df: pd.DataFrame) -> pd.DataFrame:
    """SA_result 열에서 감성 점수(prob)와 라벨(label) 추출"""
    df['sentiment_score'] = df['SA_result'].apply(lambda x: x['prob'])
    df['sentiment_label'] = df['SA_result'].apply(lambda x: x['label'])
    return df

def filter_by_percentile_and_label(df: pd.DataFrame, label: str, percentile: int = 20) -> pd.DataFrame:
    """
    특정 감정(label)의 점수 상위 percentile 필터링
    """
    # 특정 감정(label) 필터링
    filtered_df = df[df['sentiment_label'] == label]
    
    # 상위 percentile 계산
    upper = filtered_df['sentiment_score'].quantile(1 - percentile / 100)
    
    # 상위 percentile 데이터 반환
    significant_news = filtered_df[filtered_df['sentiment_score'] >= upper]
    
    return significant_news

# sector_info에서 필요한 항목만 추출하는 코드
def extract_key_metrics(sector_info):
    extracted_info = {
        'Adj. R-squared': sector_info.get('Adj. R-squared'),
        'F-statistic': sector_info.get('F-statistic'),
        'Prob (F-statistic)': sector_info.get('Prob (F-statistic)'),
        'Durbin-Watson': sector_info.get('Durbin-Watson'),
        'Omnibus': sector_info.get('Omnibus'),
        'Prob(JB)': sector_info.get('Prob(JB)'),
    }

    # 주요 요인별 정보 추출
    for key in ['const_result', 'R_m_minus_R_f_result', 'SMB_result', 'HML_result', 'MOM_result']:
        if key in sector_info:
            extracted_info[key] = {
                'coef': sector_info[key].get(f'{key.split("_result")[0]}_coef'),
                'P_t': sector_info[key].get(f'{key.split("_result")[0]}_P_t'),
                'confid_interval': sector_info[key].get(f'{key.split("_result")[0]}_confid_interval'),
            }
    
    return extracted_info

### 프롬프트 모음

In [19]:
f_system = f"""당신은 증권회사에 고용된 재무전문가입니다. 
주식투자의 관점에서 주어진 정보들을 요약하고, 이에 대한 의견을 알려주세요. 
의견을 개진할 때에는 반드시 보고서에서 근거를 제시해야 합니다.

응답은 반드시 markdown 문법에 따라 작성되어야 합니다.

** 반드시 markdown 형식으로 작성하세요 **
** 보고서에는 반드시 주어진 정보에 대한 분석이 필요합니다 **
"""

f_prompt = f""""""

# ============================ #

intl_macro_system = f"""당신은 증권회사에 고용된 국제관계전문가입니다.
주식투자의 관점에서 주어진 정보들을 요약하고, 이에 대한 의견을 알려주세요.
국가별로 금리 동향, GDP, 인플레이션 등 주식투자에 필요한 거시경제 정보를 종합하여 레포트를 작성해야 합니다.

응답은 반드시 markdown 문법에 따라 작성되어야 합니다.

** 반드시 markdown 형식으로 작성하세요 **
** 보고서에는 반드시 주어진 정보에 대한 분석이 필요합니다 **
"""

intl_macro_prompt = f""""""

# ============================ #

index_sector_system = f"""당신은 증권회사에 고용된 경제전문가입니다.
주식투자의 관점에서 주어진 정보들을 요약하고, 이에 대한 의견을 알려주세요.
산업군별 투자 매력도를 평가하고, 보고서에 근거를 제시해야 합니다..

응답은 반드시 markdown 문법에 따라 작성되어야 합니다.

** 반드시 markdown 형식으로 작성하세요 **
** 보고서에는 반드시 주어진 정보에 대한 분석이 필요합니다 **
"""

index_sector_prompt = f""""""

# ============================ #

t_1_analyst_system = f"""당신은 증권회사 애널리스트팀의 리더입니다.
팀원들이 투자에 필요한 정보들에 대해 레포트를 작성하였습니다.
보고받은 자료에 기반해 포트폴리오를 어떻게 조정해야 할지, 오늘의 투자전략에 대한 전반적인 제안서를 작성하세요.
"""

t_1_analyst_prompt = f""""""

### 종목 무관

In [ ]:
# 국제 뉴스
intl_title_list = list(intl_news_info(target_year, start_date, end_date)['news_title'])

# === 거시 경제 관련 정보 수집 === #
def create_macro_econ_dict(country, econ_item):
    result_dict = macro_econ_dict[country][econ_item].set_index('Date').to_dict('index')
    final_dict = {k: v[list(v.keys())[0]] for k, v in result_dict.items()}

    return final_dict

macro_econ_dict = macro_econ_info(target_year, start_date, end_date)
country_list = list(macro_econ_dict.keys())

reports_dict = {}

for country in country_list:

    target_country_dict = macro_econ_dict[country]
    econ_items = list(target_country_dict.keys())

    for econ_item in econ_items:
        final_dict = create_macro_econ_dict(country, econ_item)
        reports_dict[f"{country}_{econ_item}"] = final_dict

intl_macro_prompt += f"국제 뉴스 헤드라인: {intl_title_list}\n"
intl_macro_prompt += f"거시경제 관련 정보: {reports_dict}"

# ** 국제/거시경제 관련 GPT 응답 ** #
intl_gpt_response = to_GPT(intl_macro_system, intl_macro_prompt)
print('intl_gpt_response 생성 완료!')

In [20]:
# === 인덱스 가격 정보 수집 === #
index_prices = {}
sector_list = [s for s in os.listdir('../store_data/raw/market_data/sector') if '코스피' not in s]

for sector in sector_list:
    # 토큰수 제한때문에 컬럼 선별해서 넣기...
    index_price = index_price_info(sector, start_date, end_date)[['Close', 'Transaction_Val','Market_Cap', 'RSI_14']]
    index_prices[sector] = index_price.T.to_dict()

# === 섹터별 분석 수집 === #
sector_infos = {}

for sector in sector_list:
    sector_info = sector_analysis_info(sector, target_year, target_quarter)
    # 이것도.. 토큰수 제한때문에 컬럼 선별해서 넣기...
    sector_infos[sector] = extract_key_metrics(sector_info)

index_sector_prompt += f"섹터별 가격 정보: {index_prices}\n"
index_sector_prompt += f"섹터별 carhart 4 factor 분석: {sector_infos}"

# ** 인덱스/섹터별 GPT 응답 ** #
index_sector_response = to_GPT(index_sector_system, index_sector_prompt)
print('index_sector_response 생성 완료!')

index_sector_response 생성 완료!


### 종목 정보

In [21]:
# === 재무제표 및 재무비율 관련 정보 === #
fin_statement = get_raw_fin_statement_info(ticker, target_year, target_quarter)
fin_ratio = fin_statement_info(ticker, target_year, target_quarter)
fin_report = reports_info(ticker, target_year, target_quarter)

f_prompt += f"재무제표: {fin_statement.T.to_dict()}"
f_prompt += f"주요 재무 비율: {fin_ratio}\n"
f_prompt += f"재무보고서: {fin_report['1. 요약재무정보.csv'][0][4:-4]} "

f_gpt_response = to_GPT(f_system, f_prompt)

# === 기업 관련 뉴스 정보 === #
# 데이터 정제
corp_rel_news_df = corp_rel_news_info(ticker, target_year, start_date, end_date)
corp_rel_news_df = corp_rel_news_df[corp_rel_news_df['news_category'].str.contains('증권')]
corp_rel_news_df['SA_result'] = corp_rel_news_df['news_title'].apply(get_SA_result)

# 데이터프레임에서 감성 점수 및 라벨 추출
corp_rel_news_df = extract_sentiment_score(corp_rel_news_df)

# Positive, Negative 감정의 상위 20% 뉴스
positive_top_20 = filter_by_percentile_and_label(corp_rel_news_df, label='positive', percentile=20)
negative_top_20 = filter_by_percentile_and_label(corp_rel_news_df, label='negative', percentile=20)

# 텍스트만 남기기
positive_title_list = [positive_top_20.iloc[i]['news_title'] for i in range(len(positive_top_20))]
negative_title_list = [negative_top_20.iloc[i]['news_title'] for i in range(len(negative_top_20))]
news_title_list = {
    'Positive': positive_title_list,
    'Negative': negative_title_list
}

# === 주식 가격 정보 수집 === #
stock_price = stock_price_info(ticker, start_date, end_date).T.to_dict()

# === 차트 패턴 분석 정보 === #
# chart_pattern_info = pattern_info(ticker, end_date)

### 애널리스트 최종 output

In [22]:
t_1_analyst_prompt += f"재무제표 및 재무 비율 관련 보고서: {f_gpt_response['choices'][0]['message']['content']}\n"
t_1_analyst_prompt += f"종목 관련 뉴스 헤드라인: {news_title_list}\n"
t_1_analyst_prompt += f"종목 가격: {stock_price}\n"
# t_1_analyst_prompt += f"종목 차트 패턴 분석: {chart_pattern_info}\n"

t_1_analyst_prompt += f"국제 정세 및 거시경제 관련 정보: {intl_gpt_response['choices'][0]['message']['content']}\n"
t_1_analyst_prompt += f"섹터별 분석 정보: {index_sector_response['choices'][0]['message']['content']}\n"

In [23]:
t_1_analyst_output = to_GPT(t_1_analyst_system, t_1_analyst_prompt)

In [26]:
t_1_analyst_output['choices'][0]['message']['content'].split('\n')

['# 오늘의 투자전략 제안서',
 '',
 '## 1. 투자 환경 분석',
 '',
 '현재 글로벌 경제는 금리 인상과 인플레이션 우려 속에서 고용과 소비가 회복세를 보이고 있습니다. 미국의 GDP 성장률이 5.6%로 예상되고 있는 가운데, 금리 인상이 논의되고 있어 주식 시장의 변동성이 클 것으로 보입니다. 미국 외에도 중국, 유럽, 일본 등 각국의 경제 회복이 진행되고 있으며, 이들 국가의 통화 정책과 경제 지표도 주식 시장에 영향을 미칩니다.',
 '',
 '특히, 최근 재무제표 분석 결과에서는 특정 기업의 유동자산 증가와 현금성 자산 비중이 높아진 점이 긍정적인 요소로 작용할 수 있지만, 높은 PER과 낮은 ROE 등으로 인해 주가가 과대평가될 가능성도 존재합니다. 이와 함께 섹터별로도 다양한 투자 매력도가 존재하므로, 종합적인 접근이 필요합니다.',
 '',
 '## 2. 포트폴리오 조정 제안',
 '',
 '### 2.1. 포트폴리오 구성 조정',
 '- **방어적 자산 비중 확대**: 금리 인상과 인플레이션 우려로 인해 방어적 자산으로의 이동이 필요합니다. 특히, **음식료품**, **유통업**, **전기가스업** 등 안정적인 성과를 보이는 섹터에 비중을 두는 것이 바람직합니다.',
 '  ',
 '- **성장성 높은 섹터 투자**: **화학**, **제조업**, **철강금속**, **비금속광물** 섹터는 안정적인 상승세를 보이고 있으며, 투자 매력도가 높습니다. 이들에 대한 비중을 확대하겠습니다.',
 '',
 '- **주요 기업 주식**: 삼성전자와 SK하이닉스 등 주요 기업의 주식을 고려하되, 이들 주식의 현재 주가가 과대평가된 점을 고려하여 신중한 접근이 필요합니다. 특히, 삼성전자는 목표가 상향 조정과 긍정적인 실적 전망이 있으나, 최근 정치적 이슈로 인한 변동성이 존재합니다.',
 '',
 '### 2.2. 리스크 관리',
 '- **분산 투자**: 각 섹터에 대한 투자 비중을 고르게 분산하여 리스크를 최소화합니다. 특히, 과매수 경향

In [30]:
index_sector_response['choices'][0]['message']['content'].split('\n')

['# 섹터별 투자 매력도 분석 보고서',
 '',
 '## 1. 서론',
 '이번 보고서는 주어진 섹터별 가격 정보와 Carhart 4 Factor 분석 결과를 기반으로 각 산업군의 투자 매력도를 평가하고자 합니다. 주식 투자의 관점에서 각 섹터의 성과를 분석하여 투자자에게 유용한 정보를 제공할 것입니다.',
 '',
 '## 2. 섹터별 가격 정보 요약',
 '',
 '### 2.1. 금융업',
 '- **시장 점유율**: 21.12조 원',
 '- **가격 추세**: 안정적인 상승세를 보였으나 최근 며칠간 조정세 발생',
 '- **RSI**: 52.74 (중립)',
 '',
 '### 2.2. 증권',
 '- **시장 점유율**: 2.56조 원',
 '- **가격 추세**: 전반적으로 하락세',
 '- **RSI**: 48.79 (중립)',
 '',
 '### 2.3. 운수창고업',
 '- **시장 점유율**: 3.39조 원',
 '- **가격 추세**: 초기 상승세 후 조정',
 '- **RSI**: 68.61 (과매수 경향)',
 '',
 '### 2.4. 보험',
 '- **시장 점유율**: 3.68조 원',
 '- **가격 추세**: 안정적이나 상승폭이 적음',
 '- **RSI**: 45.93 (중립)',
 '',
 '### 2.5. 섬유의복',
 '- **시장 점유율**: 0.54조 원',
 '- **가격 추세**: 하락세',
 '- **RSI**: 54.22 (중립)',
 '',
 '### 2.6. 화학',
 '- **시장 점유율**: 220.33조 원',
 '- **가격 추세**: 상승세',
 '- **RSI**: 73.23 (과매수 경향)',
 '',
 '### 2.7. 유통업',
 '- **시장 점유율**: 72.12조 원',
 '- **가격 추세**: 지속적인 상승',
 '- **RSI**: 71.18 (과매수 경향)',
 '',
 '### 2.8. 제조업',
 '- **시장 점유율**: 1,430.92조 원',
 '- **가격 추세